In [3]:
import numpy as np
import pandas as pd

In [4]:
allCrimal_df = pd.read_csv('allcrime1227.csv')

In [7]:
print(allCrimal_df.iloc[0:5])

   age  edu_level  race  gender  labels              laws  \
0    1          3     0       0      13  中华人民共和国刑法第一百三十三条   
1    2          2     0       1     149  中华人民共和国刑法第三百八十二条   
2    0          6     0       0      72  中华人民共和国刑法第三百四十七条   
3    1          7     0       0     189  中华人民共和国刑法第二百六十四条   
4    2          5     0       0      11  中华人民共和国刑法第三百五十四条   

                                                fact  
0  2018年4月17日21时1 8分许，被告人崔某某驾驶浙BJ3***号牌小型普通客车沿本市海...  
1  2011年4月，南京市浦口区盘城街道永锦北路项目征地拆迁过程中，被告人徐金凤作为**街道征地...  
2  2017年10月至2018年1月期间，被告人张某某从他人处购买甲基苯丙胺片剂（俗称“麻果”）...  
3  1.2018年2月18日15时许,被告人张某甲、张某乙经预谋，趁成都市锦江区较场坝东街53号...  
4  1.2018年1月中旬某日15时许，被告人陈某某在自己位于安乡县黄山**的家中容留吸毒人员沈...  


In [17]:
traffic2 = allCrimal_df[allCrimal_df['labels'].isin([152,13])]

In [18]:
len(traffic2)

12396

In [21]:
traffic2.groupby(['labels']).size()

labels
13     2739
152    9657
dtype: int64

In [29]:
from keras.preprocessing.text import Tokenizer
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
def vec2bow(trafic_vec):
    tokenizer = Tokenizer(num_words=2000)
    tokenizer.fit_on_texts(trafic_vec)
    trafic_vec_seq = tokenizer.texts_to_sequences(trafic_vec)
    trafic_fact = sequence.pad_sequences(trafic_vec_seq,maxlen=150)
    return trafic_fact

In [30]:
traffic2fact = tokenization(traffic2['fact'].values)

Building prefix dict from the default dictionary ...


Dumping model to file cache C:\Users\wangfy\AppData\Local\Temp\jieba.cache


Loading model cost 2.042 seconds.


Prefix dict has been built succesfully.


In [36]:
traffic2vec = vec2bow(traffic2fact)

In [38]:
type(traffic2vec[0])

numpy.ndarray

In [45]:
def similarity(vector1,vector2):
    return np.dot(vector1,vector2)/(np.linalg.norm(vector1)*(np.linalg.norm(vector2)))

In [69]:
simall = np.zeros((len(traffic2vec),len(traffic2vec)))

In [70]:
simall.shape

(12396, 12396)

In [72]:
simtest = np.zeros((10,len(traffic2vec)))

In [73]:
import datetime
for v1 in range(10):
    start = datetime.datetime.now()
    for v2 in range(12396):
        simtest[v1,v2] = similarity(traffic2vec[v1],traffic2vec[v2])
    end = datetime.datetime.now()
    print("vector{}---time{}".format(v1,end-start))

vector0---time0:00:00.373769


vector1---time0:00:00.543665


vector2---time0:00:00.593633


vector3---time0:00:00.366775


vector4---time0:00:00.378766


vector5---time0:00:00.367774


vector6---time0:00:00.361777


vector7---time0:00:00.361778


vector8---time0:00:00.371771


vector9---time0:00:00.373769


In [122]:
simtest.mean(1)

array([0.23466291, 0.24675562, 0.2131649 , 0.23400859, 0.22175356,
       0.2747693 , 0.27608371, 0.23045448, 0.27446992, 0.27619823])

In [124]:
import matplotlib.pyplot as plt
plt.scatter(range(simtest.shape[1]),simtest[6,:])
plt.show()

In [117]:
import matplotlib.pyplot as plt
plt.scatter(range(500),simtest[9,:500])
plt.show()

In [121]:
traffic2.iloc[0]

age                                                          1
edu_level                                                    3
race                                                         0
gender                                                       0
labels                                                      13
laws                                          中华人民共和国刑法第一百三十三条
fact         2018年4月17日21时1 8分许，被告人崔某某驾驶浙BJ3***号牌小型普通客车沿本市海...
Name: 0, dtype: object

In [131]:
L=[('b',2),('a',1),('a',2),('a',3),('c',3),('d',4)]

In [133]:
sorted(L,key = lambda k:k[0])

[('a', 1), ('a', 2), ('a', 3), ('b', 2), ('c', 3), ('d', 4)]

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.linalg import norm
 
def tfidf_similarity(s1, s2):
    def add_space(s):
        return ' '.join(list(s))
    
    # 将字中间加入空格
    s1, s2 = add_space(s1), add_space(s2)
    # 转化为TF矩阵
    cv = TfidfVectorizer(tokenizer=lambda s: s.split())
    corpus = [s1, s2]
    vectors = cv.fit_transform(corpus).toarray()
    # 计算TF系数
    return np.dot(vectors[0], vectors[1]) / (norm(vectors[0]) * norm(vectors[1]))
 
s1 = '你在干啥子呢'
s2 = '你在干什么呢'
print(tfidf_similarity(s1, s2))

0.5031026124151314


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout, Input, concatenate
from keras.layers.embeddings import Embedding
from keras.models import Model

def text_cnn(maxlen=150, max_features=2000, embed_size=1):
    # Inputs
    comment_seq = Input(shape=[maxlen], name='x_seq')
    print(type(comment_seq))
    # Embeddings layers
    emb_comment = Embedding(max_features, embed_size)(comment_seq)
    # conv layers
    convs = []
    filter_sizes = [2, 3, 4, 5]
    for fsz in filter_sizes:
        l_conv = Conv1D(filters=100, kernel_size=fsz, activation='relu')(emb_comment)
        # l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)show
        l_pool = MaxPooling1D(pool_size=2)(l_conv)

        l_pool = Flatten()(l_pool)
        convs.append(l_pool)
    merge = concatenate(convs, axis=1)

    out = Dropout(0.5)(merge)
    output = Dense(32, activation='relu')(out)
    output = Dense(units=203, activation='softmax')(output)

    model = Model([comment_seq], output)
    #     adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
    return model